In [1]:
import pandas as pd

In [6]:
data_path = "Task_B_Dataset.csv"
df = pd.read_csv(data_path)
df.head()

,filename,para_id,sent_id,raw_text
0,cl02,28,1,"I/ppss couldn't/md* see/vb him/ppo ,/, but/cc ..."
1,cj10,9,2,Its/pp$ life/nn history/nn is/bez much/ql simp...
2,cb05,41,3,"And/cc the/at pay/nn ,/, of/in course/nn ,/, w..."
3,ck03,11,16,The/at only/ap one/cd who/wps would/md have/hv...
4,cj65,1,4,And/cc though/cs in/in his/pp$ later/jjr years...


In [3]:
import re
import json

def extract_phrases(pos_tags):
    phrases = []
    i = 0
    while i < len(pos_tags):
        if pos_tags[i] == "in":
            j = i + 1
            while j < len(pos_tags) and pos_tags[j] == "nn":
                j += 1
                
            if j > i + 1:
                phrase = " ".join(pos_tags[i:j])
                phrases.append((i, j - 1, phrase))  
            i = j
        else:
            i += 1

    largest_phrases = []
    if phrases:
        max_end = -1
        for start, end, phrase in phrases:
            if start > max_end:
                largest_phrases.append((start, end, phrase))
                max_end = end

    return largest_phrases


def generate_output(df):
    output = {
        "pattern": "pattern 1",
        "sents": []
    }

    for _, row in df.iterrows():
        filename = row["filename"]
        para_id = row["para_id"]
        sent_id = row["sent_id"]
        raw_text = row["raw_text"]

        # Parsing sentence into (word, pos) pairs
        tokens = raw_text.split()
        words = [t.split("/")[0] for t in tokens]
        pos_tags = [t.split("/")[1] for t in tokens]

        # Extract phrases
        matches = extract_phrases(pos_tags)

        # Sentence entry
        sentence_entry = {
            "filename": filename,
            "para_id": para_id,
            "sent_id": sent_id,
            "sent_text": " ".join(words),
            "phrases": []
        }

        # Matching phrases
        for start, end, phrase in matches:
            begin_idx = len(" ".join(words[:start])) + (1 if start > 0 else 0)
            end_idx = len(" ".join(words[:end + 1]))
            sentence_entry["phrases"].append({
                "begin": begin_idx,
                "end": end_idx,
                "text": " ".join(words[start:end + 1]),
                "phrase_type": phrase
            })

        if sentence_entry["phrases"]:
            output["sents"].append(sentence_entry)

    return output

result_json = generate_output(df) 

In [4]:
with open("pattern1.json", "w") as fp:
    json.dump(result_json , fp, indent = 4) 

In [5]:
with open("pattern1.json", "r") as fp:
    pattern1_json = json.load(fp)   
pattern1_json

{'pattern': 'pattern 1',
 'sents': [{'filename': 'cb05',
   'para_id': 41,
   'sent_id': 3,
   'sent_text': 'And the pay , of course , will be nil .',
   'phrases': [{'begin': 14,
     'end': 23,
     'text': 'of course',
     'phrase_type': 'in nn'}]},
  {'filename': 'ce35',
   'para_id': 1,
   'sent_id': 1,
   'sent_text': "It's a lot quicker and easier to dimension the kitchen to fit the cabinets and erect the end wall after they are all in place .",
   'phrases': [{'begin': 117,
     'end': 125,
     'text': 'in place',
     'phrase_type': 'in nn'}]},
  {'filename': 'cj32',
   'para_id': 2,
   'sent_id': 5,
   'sent_text': 'dictionary information about the form stored at this location can be retrieved directly by occurrences of the form in text .',
   'phrases': [{'begin': 115,
     'end': 122,
     'text': 'in text',
     'phrase_type': 'in nn'}]},
  {'filename': 'cl02',
   'para_id': 56,
   'sent_id': 0,
   'sent_text': "The place wasn't particularly busy at that time of night , 